In [139]:
%matplotlib inline
import pandas as pd
from pandas import set_option
#Limit dataframe exibition to 5 rows
set_option("display.max_rows", 5)
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gc
from sklearn import datasets, linear_model

In [2]:
df = pd.read_csv('./processed/prev_com_cd_ibge.csv', encoding='latin1')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
ibge = df['cd_ibge'].unique()

In [142]:
df2 = df

In [143]:
df2['dif'] = df2['vl_arrecadado'] - df2['vl_emissao']

Vamos predizer as emissões utilizando regressão linear

In [144]:
predictions_dif = {}
for codigo in ibge:
    regr = linear_model.LinearRegression(n_jobs=-1)
    X = df2.loc[df['cd_ibge']==codigo, ['ano']]
    y = df2.loc[df['cd_ibge']==codigo, ['dif']]
    regr.fit(X,y)    
    predictions_dif[codigo] = regr.predict(2017)[0][0]

In [150]:
df_dif = pd.DataFrame(columns = ['cd_ibge','dif'])
df_dif['cd_ibge'] = predictions_dif.keys()
df_dif['dif'] = predictions_dif.values()

In [152]:
predicoes_dif = pd.merge(df_dif, municipios, how='inner', on='cd_ibge')

In [156]:
predicoes_dif.sort_values(by='dif', ascending=False).head(5)

,cd_ibge,dif,estado,municipio
1585,3550308,4.496491e+10,São Paulo,São Paulo
4229,5300108,1.093944e+10,Distrito Federal,Brasília
4882,3304557,9.408826e+09,Rio de Janeiro,Rio de Janeiro
4329,3106200,3.717744e+09,Minas Gerais,Belo Horizonte
1494,4106902,3.200126e+09,Paraná,Curitiba


In [157]:
predicoes_dif.sort_values(by='dif', ascending=True).head(5)

,cd_ibge,dif,estado,municipio
2600,3547809,-1.680109e+09,São Paulo,Santo André
4315,3301702,-1.445198e+09,Rio de Janeiro,Duque de Caxias
4941,3304904,-1.430942e+09,Rio de Janeiro,São Gonçalo
4491,3548708,-1.308027e+09,São Paulo,São Bernardo do Campo
1233,3548500,-1.178660e+09,São Paulo,Santos


In [59]:
predictions_emissoes = {}
for codigo in ibge:
    regr = linear_model.LinearRegression(n_jobs=-1)
    X = df.loc[df['cd_ibge']==codigo, ['ano']]
    y = df.loc[df['cd_ibge']==codigo, ['vl_emissao']]
    regr.fit(X,y)    
    predictions_emissoes[codigo] = regr.predict(2017)[0][0]

Agora vamos  predizer as arrecadações

In [64]:
predictions_arrecadado = {}
for codigo in ibge:
    regr = linear_model.LinearRegression(n_jobs=-1)
    X = df.loc[df['cd_ibge']==codigo, ['ano']]
    y = df.loc[df['cd_ibge']==codigo, ['vl_arrecadado']]
    regr.fit(X,y)    
    predictions_arrecadado[codigo] = regr.predict(2017)[0][0]

Agora vamos montar dataframes para facilitar a manipulação dos dados

In [95]:
df_arrec = pd.DataFrame(columns = ['cd_ibge','vl_arrecadado'])
df_arrec['cd_ibge'] = predictions_arrecadado.keys()
df_arrec['vl_arrecadado'] = predictions_arrecadado.values()

In [96]:
df_emis = pd.DataFrame(columns = ['cd_ibge','vl_emissao'])
df_emis['cd_ibge'] = predictions_emissoes.keys()
df_emis['vl_emissao'] = predictions_emissoes.values()

Juntando os 2 dataframes

In [97]:
predicoes = pd.merge(df_arrec, df_emis, how='inner', on='cd_ibge')

Adicionando informações de estado e cidade

In [130]:
municipios = pd.DataFrame(df.groupby(by=['cd_ibge','estado','municipio']).first()).reset_index()

In [131]:
municipios.drop(['ano','vl_emissao','vl_arrecadado'], axis=1, inplace=True)

In [134]:
predicoes = pd.merge(predicoes, municipios, how='inner', on='cd_ibge')

In [137]:
predicoes['dif'] = predicoes['vl_arrecadado'] - predicoes['vl_emissao']

In [138]:
predicoes['indice'] = predicoes['vl_arrecadado'] / predicoes['vl_emissao']

In [140]:
predicoes

,cd_ibge,vl_arrecadado,vl_emissao,estado,municipio,dif,indice
0,3506201,-8.483537e+03,5.518920e+06,São Paulo,Bento de Abreu,-5.527404e+06,-0.001537
1,4128807,-2.046312e+04,6.151097e+06,Paraná,Xambrê,-6.171560e+06,-0.003327
...,...,...,...,...,...,...,...
5568,3145703,-2.266971e+03,8.026619e+05,Minas Gerais,Oliveira Fortes,-8.049289e+05,-0.002824
5569,5210109,9.913561e+06,5.248251e+07,Goiás,Ipameri,-4.256895e+07,0.188893


In [141]:
df.loc[df['cd_ibge']==3506201]

,municipio,estado,ano,vl_emissao,cd_ibge,vl_arrecadado
3336,Bento de Abreu,São Paulo,2000,783918,3506201,413520
8906,Bento de Abreu,São Paulo,2001,883695,3506201,392017
...,...,...,...,...,...,...
86886,Bento de Abreu,São Paulo,2015,4733895,3506201,0
92456,Bento de Abreu,São Paulo,2016,5392249,3506201,0
